In [10]:
import pandas as pd
import bs4
from bs4 import BeautifulSoup  
import csv
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import re

In [11]:
df = pd.read_csv("Data/Nadlan.csv")

In [12]:

def rename_cols_update_data_types(df):
    df = df.dropna(subset=['DEALAMOUNT','DEALNATURE']).reset_index(drop=True)
#     df[['NEIGHBORHOOD','NEIGHBORHOODID','ASSETMETER']] =  df[['NEIGHBORHOOD','NEIGHBORHOODID','ASSETMETER']].fillna(0)
    
    
    types = ["nan","מיני פנטהאוז","מגורים","בית בודד","דופלקס","קוטג' חד משפחתי","קוטג' דו משפחתי",'מלונאות','חנות','קרקע למגורים','קבוצת רכישה - קרקע מגורים','None','אופציה','קבוצת רכישה - קרקע מסחרי','חניה','מסחרי + מגורים','דירת נופש','דיור מוגן','קומבינציה','מבנים חקלאיים','תעשיה','מסחרי + משרדים','בניני ציבור','חלוקה/יחוד דירות','מחסנים','אחר','בית אבות','עסק',"קוטג' טורי",'ניוד זכויות בניה','משרד','ללא תיכנון','מלונאות ונופש','משרדים + מגורים','מלאכה',]
    df = df.drop(df[df['DEALNATUREDESCRIPTION'].isin(types)].index).reset_index(drop=True)
    
    df = split_gush_helka_tat(df)
    df = split_street_homenumber(df)
    df = convert_price(df)
    df = convert_date_type(df)
    

    df.rename(columns = {'BUILDINGYEAR':'Build_year'}, inplace = True)
    df['Build_year'] = df['Build_year'].fillna(value = '0')
    df['Build_year'] = df['Build_year'].astype(int)

    df.rename(columns = {'ASSETROOMNUM':'Rooms'}, inplace = True)
           
    df.rename(columns = {'DEALNATURE':'Size'}, inplace = True)
    df['Size'] = df['Size'].fillna(value = '0')
    df['Size'] = df['Size'].astype(int)
    
    df.rename(columns = {'DEALNATUREDESCRIPTION':'Type'}, inplace = True)
    df.rename(columns = {'NEWPROJECTTEXT':'New'}, inplace = True)
    df.rename(columns = {'YEARBUILT':'Rebuilt'}, inplace = True)
    df.rename(columns = {'BUILDINGFLOORS':'Floors'}, inplace = True)

    cols= ["DEALDATETIME","FULLADRESS","PROJECTNAME","TREND_IS_NEGATIVE","TREND_FORMAT",'KEYVALUE','POLYGON_ID','TYPE']

    df = df.drop(columns=cols, axis=1)
    return df

def split_gush_helka_tat(df):
    df[['Gush', 'Helka','Tat']] = df['GUSH'].str.split('-|/', n=2, expand=True).astype(int)  
    df = df.drop(columns='GUSH', axis=1)
    return df

def split_street_homenumber(df):
    # Extract the Home_number
#     df['DISPLAYADRESS'] = df['DISPLAYADRESS'].fillna(value = 'NO0')
    df = df.dropna(subset=['DISPLAYADRESS']).reset_index(drop=True)
    df['Home_number'] = df['DISPLAYADRESS'].str.extract('([0-9]+)', expand=False)
    df['Home_number'] = pd.to_numeric(df['Home_number'], errors='coerce').astype('Int64')

#     df['Home_number'] = np.where(df['Home_number'].isnull(), np.nan, df['Home_number'].astype(int))
    
    # Extract the Street
    df['DISPLAYADRESS'] = df['DISPLAYADRESS'].str.replace('\d+', '').str.strip()
    df = df.rename(columns={'DISPLAYADRESS': 'Street'})
    return df  

def convert_date_type(df):
    df.rename(columns = {'DEALDATE':'Date'}, inplace = True)
    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%d.%m.%Y')
    return df

def convert_price(df):
    df.rename(columns = {'DEALAMOUNT':'Price'}, inplace = True)
    df['Price'] = df['Price'].str.replace(',', '').astype(int)
    return df

df = rename_cols_update_data_types(df)

<ipython-input-12-30aca36664c3>:50: FutureWarning: The default value of regex will change from True to False in a future version.
  df['DISPLAYADRESS'] = df['DISPLAYADRESS'].str.replace('\d+', '').str.strip()


In [13]:
floors = {-1:'מרתף', 
     0:'קרקע', 
     1:'ראשונה', 
     2:'שניה', 
     3:'שלישית', 
     4:'רביעית', 
     5:'חמישית', 
     6:'שישית', 
     7:'שביעית', 
     8:'שמינית', 
     9:'תשיעית', 
     10:'עשירית', 
     11:'אחת עשרה', 
     12:'שתים עשרה', 
     13:'שלוש עשרה', 
     14:'ארבע עשרה', 
     15:'חמש עשרה', 
     16:'שש עשרה', 
     17:'שבע עשרה',  
     18:'שמונה עשרה', 
     19:'תשע עשרה', 
     20:'עשרים', 
     21:'עשרים ואחת', 
     22:'עשרים ושתיים', 
     23:'עשרים ושלוש', 
     24:'עשרים וארבע', 
     25:'עשרים וחמש', 
     26:'עשרים ושש', 
     27:'עשרים ושבע', 
     28:'עשרים ושמונה', 
     29:'עשרים ותשע',
     30:'שלושיים',
     31:'שלושיים ואחת',
     32:'שלושיים ושתיים',
     33:'שלושים ושלוש',
     34:'שלושים וארבע',
     35:'שלושים וחמש',
     36:'שלושים ושש',
     37:'שלושים ושבע',
     38:'שלושים ושמונה',
     39:'שלושים ותשע',
     40:'ארבעים',
    }


def floor_to_numeric(df, floors):
    floor_dict = {value: key for key, value in floors.items()}
    for floor, num in floor_dict.items():
        df.loc[df['FLOORNO'] == floor, 'FLOORNO'] = num
    df = df[df['FLOORNO'].isin(list(floor_dict.values()))].reset_index(drop=True)
    df.rename(columns = {'FLOORNO':'Floor'}, inplace = True)

    return df

df = floor_to_numeric(df,floors)
print(df.shape)

(64716, 15)


In [14]:
df_b = pd.read_csv("Data/Buildings.csv")
df_a = pd.read_csv("Data/Addresses.csv")


In [15]:
def edit_addresses(df):
    df = df.dropna(subset=['ms_gush','ms_chelka']).reset_index(drop=True)
    df.rename(columns = {'ms_gush':'Gush'}, inplace = True)
    df['Gush'] = df['Gush'].astype(int)
    
    df.rename(columns = {'ms_chelka':'Helka'}, inplace = True)
    df['Helka'] = df['Helka'].astype(int)
    
    df.rename(columns = {'t_rechov':'Street'}, inplace = True)
    df.rename(columns = {'ms_bayit':'Home_number'}, inplace = True)

    return df


df_a = edit_addresses(df_a)

In [16]:
def megre_df(df1,df2):
    df1 = df1.dropna(subset=['Home_number','Street']).reset_index(drop=True)
    df2 = df2.dropna(subset=['Home_number','Street']).reset_index(drop=True)
    
    df1['Gush'] = df1['Gush'].astype(str)
    df1['Helka'] = df1['Helka'].astype(str)
    df1['Home_number'] = df1['Home_number'].astype(str)


    df2['Gush'] = df2['Gush'].astype(str)
    df2['Helka'] = df2['Helka'].astype(str)
    df2['Home_number'] = df2['Home_number'].astype(str)
    
    df1['m'] = df1['Gush'] + df1['Helka'] + df1['Home_number']
    df2['m'] = df2['Gush'] + df2['Helka'] + df2['Home_number']
    
    df2 = df2.drop(columns=['Street','Home_number','Gush','Helka'], axis=1)
    df2.drop_duplicates(subset =['m'], inplace =True)
    merged = pd.merge(df1, df2, on=['m'],how='left')
    
    cols = ['m','t_ktovet_melea','t_bayit_veknisa','knisa','k_status_hesder','k_rechov','lon','lat','id_ktovet']
    merged = merged.drop(columns=cols, axis=1)
    
    merged.rename(columns = {'x':'Long'}, inplace = True)
    merged.rename(columns = {'y':'Lat'}, inplace = True)

    return merged

df_m = megre_df(df,df_a)
# df_a['m'].is_unique
# a[['d', 'id_binyan']] = a['UniqueId'].str.split('-', n=1, expand=True)


In [17]:
def add_neighborhoods():
    df = pd.read_csv("Data/Real_Estate_TLV_GOVMAPS_1.csv",index_col=0)
    neighborhoods = {}
    for index, row in df.iterrows():
        neighborhood = row['NEIGHBORHOOD']
        street = row['STREENNAME']
        if neighborhood not in neighborhoods:
            neighborhoods[neighborhood] = set()
        neighborhoods[neighborhood].add(street)
    return neighborhoods

def add_neighborhood_column(df, neighborhoods):
    # Create a new column called 'neighborhood'
    df['Neighborhood'] = ''
    
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        street = row['Street']
        
        # Loop through each neighborhood in the neighborhoods dictionary
        for neighborhood, streets in neighborhoods.items():
            # Check if the street is in the set of streets for the neighborhood
            if street in streets:
                # If it is, assign the neighborhood to the 'neighborhood' column for this row
                df.at[index, 'Neighborhood'] = neighborhood
                break # Break out of the inner loop once we find a match
        
    return df

df_m = add_neighborhood_column(df_m,add_neighborhoods())
df_m = df_m.dropna(subset=['Neighborhood']).reset_index(drop=True)
df_m

,Date,Street,Type,Rooms,Floor,Size,Price,New,Build_year,Rebuilt,Floors,Gush,Helka,Tat,Home_number,Long,Lat,UniqueId,Neighborhood
0,08.03.2009,רב אשי,דירה בבית קומות,4.0,1,108,2350000,NaN,1970,NaN,8.0,6630,234,3,13,181262.55,669790.75,527-98565,נווה אביבים וסביבתה
1,08.03.2009,בני דן,דירה בבית קומות,3.0,1,70,1725000,NaN,1950,NaN,4.0,6212,998,20,2,NaN,NaN,NaN,הצפון החדש החלק הצפוני
2,08.03.2009,יונה הנביא,דירה בבית קומות,4.0,3,100,265000,NaN,1935,NaN,4.0,6915,25,12,44,178329.82,664370.23,527-70100,כרם התימנים
3,08.03.2009,בית צורי אליהו,דירה בבית קומות,4.0,1,76,1242000,NaN,1970,NaN,8.0,6631,165,98,28,181400.28,670121.86,527-99993,רמת אביב ג
4,08.03.2009,קהילת קנדה,דירה בבית קומות,2.5,2,42,300000,NaN,1970,NaN,3.0,7050,170,10,25,176787.25,659914.06,527-111597,יפו ד (גבעת התמרים
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61734,14.07.2004,"מזא""ה",דירה בבית קומות,3.0,3,69,1254000,NaN,1980,NaN,5.0,6941,165,8,60,179377.39,663624.92,527-130099,לב העיר
61735,14.07.2004,ארלוזורוב,דירה בבית קומות,3.0,3,68,735000,NaN,1960,NaN,4.0,6214,331,13,43,179152.03,665985.03,527-73562,הצפון הישן-החלק הצפוני
61736,14.07.2004,אושה,דירה בבית קומות,4.5,2,110,1123000,NaN,2020,NaN,4.0,6630,523,35,11,180782.08,669675.08,527-99656,נווה אביבים וסביבתה
61737,14.07.2004,בויאר אברהם,דירה בבית קומות,5.0,8,135,1662000,NaN,1990,NaN,11.0,6631,195,50,3,180998.17,670110.98,527-100486,רמת אביב ג


In [19]:
def calc_distance_from_the_see_TLV(X_coordinate, Y_coordinate):
    north_x = 180471
    north_y = 672391
    south_x = 177333
    south_y = 663016
    
    m = (south_y - north_y) / (south_x - north_x)   
    b = north_y - (m * north_x)
    
    numerator = abs(m * X_coordinate - Y_coordinate + b)
    denominator = math.sqrt(m**2 + 1)
    return numerator / denominator

def distance_from_sea_TLV(df):
    '''
    Need to fix long ? lat ? 
    '''
    df = df.dropna(subset=['Lat', 'Long']).reset_index(drop=True)
    latitudes = df['Long'].astype(float)
    longitudes = df['Lat'].astype(float)
    distances = [calc_distance_from_the_see_TLV(lat, lon) for lat, lon in zip(latitudes, longitudes)]
    df['Distance_sea'] = distances
    df['Distance_sea'] = df['Distance_sea'].astype(int)
    return df

df_m = distance_from_sea_TLV(df_m)
df_m

,Date,Street,Type,Rooms,Floor,Size,Price,New,Build_year,Rebuilt,Floors,Gush,Helka,Tat,Home_number,Long,Lat,UniqueId,Neighborhood,Distance_sea
0,08.03.2009,רב אשי,דירה בבית קומות,4.0,1,108,2350000,NaN,1970,NaN,8.0,6630,234,3,13,181262.55,669790.75,527-98565,נווה אביבים וסביבתה,1575
1,08.03.2009,יונה הנביא,דירה בבית קומות,4.0,3,100,265000,NaN,1935,NaN,4.0,6915,25,12,44,178329.82,664370.23,527-70100,כרם התימנים,515
2,08.03.2009,בית צורי אליהו,דירה בבית קומות,4.0,1,76,1242000,NaN,1970,NaN,8.0,6631,165,98,28,181400.28,670121.86,527-99993,רמת אביב ג,1601
3,08.03.2009,קהילת קנדה,דירה בבית קומות,2.5,2,42,300000,NaN,1970,NaN,3.0,7050,170,10,25,176787.25,659914.06,527-111597,יפו ד (גבעת התמרים,467
4,08.03.2009,בן עזרא,דירה בבית קומות,2.0,2,52,1340000,NaN,1970,NaN,3.0,6953,30,13,5,179374.62,665694.23,527-75923,הצפון הישן-החלק הצפוני,1085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58940,14.07.2004,"מזא""ה",דירה בבית קומות,3.0,3,69,1254000,NaN,1980,NaN,5.0,6941,165,8,60,179377.39,663624.92,527-130099,לב העיר,1745
58941,14.07.2004,ארלוזורוב,דירה בבית קומות,3.0,3,68,735000,NaN,1960,NaN,4.0,6214,331,13,43,179152.03,665985.03,527-73562,הצפון הישן-החלק הצפוני,782
58942,14.07.2004,אושה,דירה בבית קומות,4.5,2,110,1123000,NaN,2020,NaN,4.0,6630,523,35,11,180782.08,669675.08,527-99656,נווה אביבים וסביבתה,1157
58943,14.07.2004,בויאר אברהם,דירה בבית קומות,5.0,8,135,1662000,NaN,1990,NaN,11.0,6631,195,50,3,180998.17,670110.98,527-100486,רמת אביב ג,1223


In [126]:
# df_m.to_csv('Data/Nadlan_clean.csv') 

In [113]:
def merge_dataframes_onID(a, b):
    a = a.dropna(subset=['UniqueId'])
    a[['d', 'id_binyan']] = a['UniqueId'].str.split('-', n=1, expand=True)
    a = a.drop(columns='d', axis=1)


    a.loc[:, 'id_binyan'] = a['id_binyan'].astype(int)
    b.loc[:, 'id_binyan'] = b['id_binyan'].astype(int)
    
    merged = pd.merge(a, b, on=['id_binyan'],how='left')
    
    if merged['max_height'].isnull().any():
        # Create a new dataframe with only the rows that have NaN in 'max_height' column
        nan_rows = merged[merged['max_height'].isnull()]

        # Merge again with the original 'b' dataframe using 'id_binyan_orig' and 'id_binyan' columns
        b_subset = b[['id_binyan_orig', 'max_height']]
        b_subset.rename(columns={'id_binyan_orig': 'id_binyan'}, inplace=True)
        nan_merged = pd.merge(nan_rows, b_subset, on=['id_binyan'], how='left')

        # Combine the newly merged dataframe with the rest of the original merged dataframe
        merged.update(nan_merged)
    
    return merged

